# SplatNLP Colab Demo

Run-all notebook (~5–8 min) that demonstrates:

- Multi-label set completion in token space
- Constraint-aware decoding (beam search + exact allocator) → a legal gear build
- Sparse Autoencoder (SAE) feature readouts (probe mode) + token influence

Tip: switch to a GPU runtime if available; CPU works but is slower.


In [ ]:
import os
import subprocess
from pathlib import Path

REPO_URL = "https://github.com/cesaregarza/SplatNLP.git"
REPO_DIR = Path("/content/SplatNLP")

if not REPO_DIR.exists():
    subprocess.run(
        ["git", "clone", "--depth", "1", REPO_URL, str(REPO_DIR)],
        check=True,
    )

os.chdir(REPO_DIR)
print("Repo ready:", REPO_DIR)


In [ ]:
%%capture
!pip -q install -e .


In [ ]:
import splatnlp

print("✅ Installed splatnlp (editable)")
print("splatnlp import:", splatnlp.__file__)


## Model at a glance

SplatGPT is a SetTransformer-style model: embed ability tokens + weapon ID,
process the (unordered) set with attention layers, pool to a 512-D vector,
then predict every token independently (multi-label).


In [ ]:
from pathlib import Path

from IPython.display import Image, display

img = Path("neural_network_themed.gv.png")
if img.exists():
    display(Image(filename=str(img), width=460))
else:
    print("Diagram not found:", img)


## Download artifacts (Ultra by default)

This grabs `model_params.json`, `vocab.json`, `weapon_vocab.json`, and
`model_info.json`, plus the model checkpoint.

- `USE_ULTRA_MODEL=True` uses the Ultra checkpoint (`model_ultra.pth`).
- `ENABLE_ULTRA_SAE=True` also downloads the Ultra SAE (and optional labels)
  so the mech-interp section can run.

Optional: if feature labels are available in the dataset directory
(`feature_labels_ultra.json` or `consolidated_ultra.json`), the Ultra section
will display human-readable names for top SAE features.


In [ ]:
from pathlib import Path

from splatnlp.utils.download_artifacts import (
    CORE_ARTIFACTS,
    ULTRA_ARTIFACTS,
    ULTRA_LABELS_ARTIFACTS,
    ULTRA_SAE_ARTIFACTS,
    download_artifacts,
)

BASE_URL = "https://splat-nlp.nyc3.cdn.digitaloceanspaces.com"
DATASET_DIR = "dataset_v2"
OUT_DIR = Path("saved_models") / DATASET_DIR

USE_ULTRA_MODEL = True
ENABLE_ULTRA_SAE = True
FORCE_DOWNLOAD = False
VERBOSE_LIST = False

# If we enable the Ultra SAE, we must use the Ultra backbone.
if ENABLE_ULTRA_SAE:
    USE_ULTRA_MODEL = True

MODEL_FILENAME = "model_ultra.pth" if USE_ULTRA_MODEL else "model.pth"

artifacts = list(CORE_ARTIFACTS)
if USE_ULTRA_MODEL:
    artifacts = [a for a in artifacts if a.filename != "model.pth"]
    artifacts += list(ULTRA_ARTIFACTS)

if ENABLE_ULTRA_SAE:
    # SAE implies Ultra model.
    if not USE_ULTRA_MODEL:
        artifacts += list(ULTRA_ARTIFACTS)
    artifacts += list(ULTRA_SAE_ARTIFACTS) + list(ULTRA_LABELS_ARTIFACTS)

# De-dup by filename (preserve order).
deduped = []
seen = set()
for spec in artifacts:
    if spec.filename in seen:
        continue
    seen.add(spec.filename)
    deduped.append(spec)
artifacts = deduped

download_artifacts(
    base_url=BASE_URL,
    dataset_dir=DATASET_DIR,
    out_dir=OUT_DIR,
    artifacts=artifacts,
    force=FORCE_DOWNLOAD,
    timeout_s=600,
    quiet=False,
    dry_run=False,
)

must_have = [
    "model_params.json",
    "vocab.json",
    "weapon_vocab.json",
    MODEL_FILENAME,
]
if ENABLE_ULTRA_SAE:
    must_have += [
        "sae_config_ultra.json",
        "sae_model_ultra.pth",
    ]

missing = [name for name in must_have if not (OUT_DIR / name).exists()]
if missing:
    raise FileNotFoundError(f"Missing artifacts after download: {missing}")

print(f"✅ Artifacts OK ({len(must_have)} required) → {OUT_DIR}")
if VERBOSE_LIST:
    for p in sorted(OUT_DIR.iterdir()):
        print(" -", p.name)


## Load model + run token-space inference

This model is **multi-label set completion**: it outputs an independent probability for every token (not an autoregressive next-token LM).

Tokenization note:
- Standard abilities are represented as cumulative threshold tokens (e.g., 12 AP `run_speed_up` → `run_speed_up_3`, `_6`, `_12`).
- Beam search below runs on capstones (highest tier per ability family), then expands them back to cumulative tokens when calling the model.


In [ ]:
import json
import random

import numpy as np
import torch

from splatnlp.model.models import SetCompletionModel
from splatnlp.serve.tokenize import tokenize_build
from splatnlp.utils.constants import (
    BUCKET_THRESHOLDS,
    MAIN_ONLY_ABILITIES,
    NULL,
    PAD,
    STANDARD_ABILITIES,
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
print("Device:", device)

# Toggle: set True to print token-space tables (useful, but verbose).
SHOW_TOKEN_TABLES = False

params = json.loads((OUT_DIR / "model_params.json").read_text())
vocab = json.loads((OUT_DIR / "vocab.json").read_text())
weapon_vocab = json.loads((OUT_DIR / "weapon_vocab.json").read_text())

pad_token_id = params.get("pad_token_id", vocab[PAD])
inv_vocab = {v: k for k, v in vocab.items()}

model = SetCompletionModel(**params)
model.load_state_dict(torch.load(OUT_DIR / MODEL_FILENAME, map_location="cpu"))
model.to(device)
model.eval()

n_params = sum(p.numel() for p in model.parameters())
print(
    f"Model params: {n_params/1e6:.1f}M | "
    f"vocab={len(vocab)} | weapons={len(weapon_vocab)}"
)

weapon_id = (
    "weapon_id_8000" if "weapon_id_8000" in weapon_vocab else next(iter(weapon_vocab))
)

partial_build = {
    "ink_saver_main": 6,
    "run_speed_up": 12,
    "intensify_action": 12,
}


def build_to_capstones(build: dict[str, int]) -> list[str]:
    capstones: list[str] = []
    for ability in MAIN_ONLY_ABILITIES:
        if build.get(ability):
            capstones.append(ability)

    for ability in STANDARD_ABILITIES:
        ap = build.get(ability)
        if ap is None:
            continue
        ap = int(ap)
        thresh = max((t for t in BUCKET_THRESHOLDS if t <= ap), default=None)
        if thresh is None:
            continue
        capstones.append(f"{ability}_{thresh}")

    return capstones or [NULL]


def predict_probs(context_tokens: list[str]) -> torch.Tensor:
    x = torch.tensor([[vocab[t] for t in context_tokens]], device=device)
    w = torch.tensor([[weapon_vocab[weapon_id]]], device=device)
    mask = x == pad_token_id
    with torch.inference_mode():
        probs = torch.sigmoid(model(x, w, key_padding_mask=mask)).squeeze(0)
    return probs.detach().cpu()


def predict_top_tokens(context_tokens: list[str], k: int = 15):
    probs_cpu = predict_probs(context_tokens)
    skip = {vocab.get(PAD), vocab.get(NULL)}
    top = torch.topk(probs_cpu, k=min(k, probs_cpu.numel()))
    out = []
    for idx, p in zip(top.indices.tolist(), top.values.tolist()):
        tok = inv_vocab[idx]
        if tok.startswith("<") or idx in skip:
            continue
        out.append((tok, float(p)))
    return out


def top_delta_tokens(delta_vec: torch.Tensor, k: int, *, largest: bool):
    skip = {vocab.get(PAD), vocab.get(NULL)}
    values, indices = torch.topk(
        delta_vec if largest else -delta_vec,
        k=min(k, delta_vec.numel()),
    )
    out = []
    for idx, v in zip(indices.tolist(), values.tolist()):
        tok = inv_vocab[idx]
        if tok.startswith("<") or idx in skip:
            continue
        dv = float(v) if largest else -float(v)
        out.append((tok, dv))
    return out


baseline_tokens = [NULL]
partial_tokens = tokenize_build(partial_build)
capstone_tokens = build_to_capstones(partial_build)

print("weapon_id:", weapon_id)
print("partial_build:", partial_build)
print("partial_tokens (cumulative):", partial_tokens)
print("capstone_tokens (for beam search):", capstone_tokens)

if SHOW_TOKEN_TABLES:
    print("\nTop predictions from <NULL> (baseline):")
    for tok, p in predict_top_tokens(baseline_tokens, k=12):
        print(f"{tok:<32} {p:.4f}")

    print("\nTop predictions from partial build tokens:")
    for tok, p in predict_top_tokens(partial_tokens, k=12):
        print(f"{tok:<32} {p:.4f}")

    baseline_probs = predict_probs(baseline_tokens)
    partial_probs = predict_probs(partial_tokens)
    delta = partial_probs - baseline_probs

    print("\nTop +Δ tokens (partial - baseline):")
    for tok, dv in top_delta_tokens(delta, k=12, largest=True):
        print(f"{tok:<32} {dv:+.4f}")

    print("\nTop -Δ tokens (partial - baseline):")
    for tok, dv in top_delta_tokens(delta, k=12, largest=False):
        print(f"{tok:<32} {dv:+.4f}")
else:
    print("\n(Set SHOW_TOKEN_TABLES=True to print token-space tables.)")


## Beam search: token-space → legal build-space

Interpretation note: this notebook shows **recommendation-style** decoding.
The decoder is free to choose any slot assignment that yields a legal build
consistent with the requested ability-point hints (it does *not* freeze specific
gear pieces).

We run two demos:

- Baseline build from `<NULL>` only
- Completion given a partial build (capstone tokens)

This uses the repo’s constraint-aware reconstruction (allocator + beam search).

Tip: set `SHOW_TRACE = True` to print a compact step-by-step trace.


In [ ]:
from splatnlp.utils.reconstruct.allocator import Allocator
from splatnlp.utils.reconstruct.beam_search import reconstruct_build

import time

allocator = Allocator()
vocab_size = len(vocab)
inv_vocab_list = [inv_vocab[i] for i in range(vocab_size)]

def predict_fn(current_tokens: list[str], weapon_id: str):
    ids = [vocab[t] for t in current_tokens]
    x = torch.tensor([ids], device=device)
    w = torch.tensor([[weapon_vocab[weapon_id]]], device=device)
    mask = x == pad_token_id
    with torch.inference_mode():
        probs = torch.sigmoid(model(x, w, key_padding_mask=mask)).squeeze(0)
    probs = probs.detach().cpu().tolist()
    return {inv_vocab_list[i]: float(probs[i]) for i in range(vocab_size)}

def summarize_trace(trace, top_preds: int = 8) -> None:
    seen = set()
    for fr in trace:
        caps = sorted(fr.partial_caps.keys())
        added = sorted(set(caps) - seen)
        seen.update(caps)
        top = sorted(fr.logits.items(), key=lambda kv: kv[1], reverse=True)
        top = [(t, round(p, 4)) for t, p in top if not t.startswith("<")][:top_preds]
        print({"step": fr.step, "added": added, "top_preds": top})

def print_build(label: str, build) -> None:
    d = build.to_dict()
    achieved = dict(
        sorted(d["achieved_ap"].items(), key=lambda kv: kv[1], reverse=True)
    )
    print(f"\n{label}")
    print("total_ap:", d["total_ap"])
    print("mains:", d["mains"])
    print("subs:", d["subs"])
    print("achieved_ap:", achieved)

SHOW_TRACE = False

def run_beam(label: str, initial_context: list[str]):
    start = time.perf_counter()
    if SHOW_TRACE:
        builds, traces = reconstruct_build(
            predict_fn=predict_fn,
            weapon_id=weapon_id,
            initial_context=initial_context,
            allocator=allocator,
            beam_size=5,
            max_steps=6,
            top_k=1,
            record_traces=True,
        )
    else:
        builds = reconstruct_build(
            predict_fn=predict_fn,
            weapon_id=weapon_id,
            initial_context=initial_context,
            allocator=allocator,
            beam_size=5,
            max_steps=6,
            top_k=1,
            record_traces=False,
        )
        traces = None

    elapsed = time.perf_counter() - start
    print(f"{label} runtime: {elapsed:.3f}s")

    if not builds:
        raise RuntimeError(f"No valid build produced for: {label}")

    build = builds[0]
    print_build(label, build)

    if SHOW_TRACE:
        summarize_trace(traces[0])

    return build

baseline_build = run_beam("Baseline build (<NULL>)", [NULL])
completion_build = run_beam("Completion from partial capstones", capstone_tokens)

def print_ap_delta(a, b, *, max_rows: int = 15) -> None:
    a_ap = a.to_dict()["achieved_ap"]
    b_ap = b.to_dict()["achieved_ap"]
    rows = []
    for ability in sorted(set(a_ap) | set(b_ap)):
        av = int(a_ap.get(ability, 0))
        bv = int(b_ap.get(ability, 0))
        if av == bv:
            continue
        rows.append((ability, av, bv, bv - av))

    rows.sort(key=lambda r: abs(r[3]), reverse=True)
    print("\nAP changes (completion - baseline):")
    for ability, av, bv, dv in rows[:max_rows]:
        print(f"{ability:<24} {av:>2} -> {bv:>2} ({dv:+d})")

def check_constraints(build, requested: dict[str, int]) -> None:
    achieved = build.to_dict()["achieved_ap"]
    ok = True
    print("\nConstraint check (completion build):")
    for ability, req in requested.items():
        got = int(achieved.get(ability, 0))
        met = got >= int(req)
        ok = ok and met
        status = "OK" if met else "FAIL"
        print(f"{ability:<24} req={req:>2} got={got:>2} {status}")
    print("all_met:", ok)

print_ap_delta(baseline_build, completion_build)
check_constraints(completion_build, partial_build)


## Build ↔ token visualization

This renders a 57-AP build as **3 mains + 9 subs**, then shows how it collapses to token-space:

- **Beam-search capstones** (highest tier per ability family)
- **Cumulative threshold tokens** (what the model actually sees)

Sub-slot ordering is not unique; this is just a display assignment.


In [ ]:
#@title Pretty build visualization (run to render)
from collections import Counter
import html as _html

from IPython.display import HTML, display

from splatnlp.serve.tokenize import tokenize_build
from splatnlp.utils.constants import (
    BUCKET_THRESHOLDS,
    MAIN_ONLY_ABILITIES,
    NULL,
    STANDARD_ABILITIES,
)

def _abbrev(token: str) -> str:
    special = {
        "ink_recovery_up": "iru",
        "ink_resistance_up": "res",
        "ink_saver_main": "ism",
        "ink_saver_sub": "iss",
        "quick_respawn": "qr",
        "quick_super_jump": "qsj",
        "run_speed_up": "rsu",
        "sub_resistance_up": "sru",
        "special_charge_up": "scu",
        "special_power_up": "spu",
        "special_saver": "ss",
        "sub_power_up": "bpu",
        "swim_speed_up": "ssu",
        "intensify_action": "ia",
    }
    base = token
    suffix = ""
    if "_" in token and token.rsplit("_", 1)[-1].isdigit():
        base, suffix = token.rsplit("_", 1)
    short = special.get(base) or "".join(w[0] for w in base.split("_"))
    return f"{short}_{suffix}" if suffix else short

def _pretty(token: str) -> str:
    return token.replace("_", " ").title()

def _ap_to_capstones(ap: dict[str, int]) -> list[str]:
    caps: list[str] = []
    for ability in MAIN_ONLY_ABILITIES:
        if int(ap.get(ability, 0) or 0) > 0:
            caps.append(ability)
    for ability in STANDARD_ABILITIES:
        val = int(ap.get(ability, 0) or 0)
        thresh = max(
            (t for t in BUCKET_THRESHOLDS if t <= val),
            default=None,
        )
        if thresh is not None:
            caps.append(f"{ability}_{thresh}")
    return caps or [NULL]

def _palette(n: int) -> list[str]:
    base = [
        "#4c78a8",
        "#f58518",
        "#54a24b",
        "#e45756",
        "#72b7b2",
        "#b279a2",
        "#ff9da6",
        "#9d755d",
        "#bab0ac",
        "#1f77b4",
        "#ff7f0e",
        "#2ca02c",
        "#d62728",
        "#9467bd",
    ]
    if n <= len(base):
        return base[:n]
    return [base[i % len(base)] for i in range(n)]

def _build_color_map(*build_dicts: dict) -> dict[str, str]:
    fams: set[str] = set()
    for d in build_dicts:
        fams.update(d.get("achieved_ap", {}).keys())
        for v in d.get("mains", {}).values():
            if v is not None:
                fams.add(v)
        fams.update(d.get("subs", {}).keys())
    fam_list = sorted(fams)
    cols = _palette(len(fam_list))
    return {fam: col for fam, col in zip(fam_list, cols)}

def _expand_subs(subs: dict[str, int]) -> list[str | None]:
    items = []
    for fam, c in sorted(
        subs.items(),
        key=lambda kv: (-int(kv[1]), kv[0]),
    ):
        items.extend([fam] * int(c))
    if len(items) < 9:
        items.extend([None] * (9 - len(items)))
    return items[:9]

def _render_build_card(title: str, build, *, colors: dict[str, str]) -> str:
    d = build.to_dict()
    mains = d["mains"]
    subs = _expand_subs(d["subs"])
    subs_by_slot = {
        "head": subs[0:3],
        "clothes": subs[3:6],
        "shoes": subs[6:9],
    }

    ap = {k: int(v) for k, v in d["achieved_ap"].items()}
    cumulative = tokenize_build(ap)
    capstones = _ap_to_capstones(ap)

    def slot_box(label: str | None, kind: str) -> str:
        if label is None:
            return f"<div class='slot {kind} empty'>empty</div>"
        col = colors.get(label, "#eee")
        display_name = _html.escape(_pretty(label))
        short = _html.escape(_abbrev(label))
        full = _html.escape(label)
        return (
            f"<div class='slot {kind}' title='{full}' "
            f"style='background:{col}30;border-color:{col}88'>"
            f"<div class='slot-name'>{display_name}</div>"
            f"<div class='slot-abbrev'>{short}</div>"
            "</div>"
        )

    gear_cols = []
    for slot_name in ["head", "clothes", "shoes"]:
        main = mains.get(slot_name)
        col_html = [
            "<div class='gear-col'>",
            f"<div class='gear-title'>{slot_name.title()}</div>",
            slot_box(main, "main"),
        ]
        for sub in subs_by_slot[slot_name]:
            col_html.append(slot_box(sub, "sub"))
        col_html.append("</div>")
        gear_cols.append("\n".join(col_html))

    main_counts = Counter([m for m in mains.values() if m is not None])
    rows = []
    for fam in sorted(ap.keys()):
        ap_val = int(ap[fam])
        m = int(main_counts.get(fam, 0))
        s = int(d["subs"].get(fam, 0))
        col = colors.get(fam, "#eee")

        toks = []
        cap = None
        if fam in MAIN_ONLY_ABILITIES:
            toks = [fam]
            cap = fam
        elif fam in STANDARD_ABILITIES:
            toks = [f"{fam}_{t}" for t in BUCKET_THRESHOLDS if t <= ap_val]
            cap = toks[-1] if toks else None

        chip_html = []
        for t in toks:
            short = _html.escape(_abbrev(t))
            full = _html.escape(t)
            cls = "chip cap" if t == cap else "chip"
            bg = f"{col}26" if t == cap else f"{col}14"
            chip_html.append(
                f"<span class='{cls}' title='{full}' "
                f"style='border-color:{col}99;background:{bg}'>{short}</span>"
            )

        rows.append(
            "\n".join(
                [
                    "<tr>",
                    (
                        "<td>"
                        f"<span class='fam' title='{_html.escape(fam)}' "
                        f"style='background:{col}22;border-color:{col}88'>"
                        f"{_html.escape(_abbrev(fam).upper())}</span>"
                        f"<span class='fam-full' title='{_html.escape(fam)}'>"
                        f"{_html.escape(_pretty(fam))}</span>"
                        "</td>"
                    ),
                    f"<td>{m}</td>",
                    f"<td>{s}</td>",
                    f"<td>{ap_val}</td>",
                    f"<td>{''.join(chip_html) if chip_html else ''}</td>",
                    "</tr>",
                ]
            )
        )

    def list_as_code(items: list[str]) -> str:
        safe = ", ".join(_html.escape(x) for x in items)
        return f"<code>{safe}</code>"

    cap_n = len([t for t in capstones if t != NULL])
    cum_n = len([t for t in cumulative if t != NULL])
    return "\n".join(
        [
            "<div class='flow-card'>",
            f"<div class='flow-title'>{_html.escape(title)}</div>",
            (
                f"<div class='small'>total_ap={int(d['total_ap'])} | "
                f"capstones={cap_n} | cumulative_tokens={cum_n}</div>"
            ),
            "<div class='gear-grid'>",
            *gear_cols,
            "</div>",
            "<details class='details'><summary>Beam-search capstones</summary>",
            list_as_code(capstones),
            "</details>",
            "<details class='details'><summary>Tokens fed to model (cumulative)</summary>",
            list_as_code(cumulative),
            "</details>",
            "<div class='small' style='margin-top:8px'>",
            "Per-family breakdown (capstone token is bold):",
            "</div>",
            "<table class='tbl'>",
            "<thead><tr><th>ability</th><th>m</th><th>s</th><th>AP</th><th>tokens</th></tr></thead>",
            "<tbody>",
            *rows,
            "</tbody></table>",
            "<div class='small note'>",
            "Note: sub-slot order is not unique; this is a display assignment.",
            "</div>",
            "</div>",
        ]
    )

STYLE = """
<style>
.flow-wrap{font-family:ui-sans-serif,system-ui,-apple-system,Segoe UI,Roboto,Arial;}
.flow-row{display:flex;gap:14px;flex-wrap:wrap;align-items:flex-start;}
.flow-card{border:1px solid #e6e6e6;border-radius:14px;padding:12px;width:520px;box-shadow:0 1px 0 rgba(0,0,0,0.02);}
.flow-title{font-weight:700;font-size:14px;margin-bottom:2px;}
.small{color:#555;font-size:12px;}
.gear-grid{display:grid;grid-template-columns:repeat(3,1fr);gap:10px;margin-top:10px;}
.gear-col{background:#fafafa;border:1px solid #eee;border-radius:12px;padding:8px;}
.gear-title{font-weight:600;font-size:12px;color:#333;margin-bottom:6px;}
.slot{border-radius:10px;padding:7px 8px;margin-top:6px;border:1px solid rgba(0,0,0,0.10);}
.slot-name{font-size:12px;font-weight:800;line-height:1.15;}
.slot.main .slot-name{font-size:13px;}
.slot.sub .slot-name{font-size:12px;font-weight:700;}
.slot-abbrev{margin-top:2px;font-size:11px;color:#222;opacity:0.85;font-family:ui-monospace,SFMono-Regular,Menlo,Monaco,Consolas,"Liberation Mono","Courier New",monospace;}
.slot.empty{background:#f0f0f0;color:#888;border-color:#e0e0e0;}
.details{margin-top:10px;}
.details summary{cursor:pointer;color:#333;font-size:12px;font-weight:600;}
code{background:#f6f6f6;border:1px solid #eee;padding:2px 6px;border-radius:8px;font-size:12px;}
.tbl{width:100%;border-collapse:collapse;margin-top:6px;}
.tbl th,.tbl td{border-bottom:1px solid #eee;padding:4px 6px;font-size:12px;vertical-align:top;}
.tbl th{color:#333;text-align:left;font-weight:700;}
.fam{display:inline-block;padding:2px 8px;border-radius:999px;border:1px solid rgba(0,0,0,0.12);font-weight:800;font-size:11px;}
.fam-full{margin-left:6px;font-size:12px;color:#222;font-weight:600;}
.chip{display:inline-block;margin:2px 4px 2px 0;padding:2px 8px;border-radius:999px;border:1px solid rgba(0,0,0,0.12);font-size:12px;}
.chip.cap{border-width:2px;font-weight:800;}
.note{margin-top:8px;}
</style>
"""

baseline_d = baseline_build.to_dict()
completion_d = completion_build.to_dict()
color_map = _build_color_map(baseline_d, completion_d)

html_out = "\n".join(
    [
        "<div class='flow-wrap'>",
        STYLE,
        "<div class='flow-row'>",
        _render_build_card("Baseline build", baseline_build, colors=color_map),
        _render_build_card("Completion build", completion_build, colors=color_map),
        "</div>",
        "</div>",
    ]
)
display(HTML(html_out))


## Ultra + SAE hook introspection (probe)

This section (optional) loads `model_ultra.pth` + `sae_model_ultra.pth`, registers
a hook on the pooled 512-D vector, then:

- checks SAE reconstruction quality (offline; model outputs unchanged)
- prints top active SAE features (optionally labeled)
- shows which tokens a feature most influences

Requires `ENABLE_ULTRA_SAE=True` in the download section above.


In [ ]:
# Toggle: set False for a faster notebook (skips SAE section).
RUN_ULTRA_SAE = bool(globals().get("ENABLE_ULTRA_SAE", True))

if not RUN_ULTRA_SAE:
    print("Skipping Ultra SAE section (RUN_ULTRA_SAE=False).")
else:
    required = [
        "model_ultra.pth",
        "sae_config_ultra.json",
        "sae_model_ultra.pth",
    ]
    missing = [name for name in required if not (OUT_DIR / name).exists()]
    if missing:
        raise FileNotFoundError(
            "Missing Ultra artifacts: "
            + ", ".join(missing)
            + ". Re-run the download cell (ENABLE_ULTRA_SAE=True)."
        )

    import json
    import textwrap

    import torch
    import torch.nn.functional as F

    from splatnlp.model.models import SetCompletionModel

    from splatnlp.monosemantic_sae.hooks import register_hooks
    from splatnlp.monosemantic_sae.models import SparseAutoencoder
    from splatnlp.utils.constants import NULL, PAD
    from splatnlp.utils.reconstruct.allocator import Allocator
    from splatnlp.utils.reconstruct.beam_search import reconstruct_build

    # Reuse the already-loaded model if it's Ultra.
    use_ultra_loaded = (
        bool(globals().get("USE_ULTRA_MODEL", False))
        and globals().get("MODEL_FILENAME") == "model_ultra.pth"
        and "model" in globals()
    )

    if use_ultra_loaded:
        ultra = model
    else:
        ultra = SetCompletionModel(**params)
        ultra.load_state_dict(
            torch.load(OUT_DIR / "model_ultra.pth", map_location="cpu")
        )
        ultra.to(device)
        ultra.eval()

    sae_cfg = json.loads((OUT_DIR / "sae_config_ultra.json").read_text())
    sae = SparseAutoencoder(
        input_dim=int(sae_cfg.get("input_dim", 512)),
        expansion_factor=float(sae_cfg.get("expansion_factor", 48.0)),
        l1_coefficient=float(sae_cfg.get("l1_coefficient", 0.0)),
        target_usage=float(sae_cfg.get("target_usage", 0.0)),
        usage_coeff=float(sae_cfg.get("usage_coeff", 0.0)),
        dead_neuron_threshold=float(sae_cfg.get("dead_neuron_threshold", 1e-6)),
        dead_neuron_steps=int(sae_cfg.get("dead_neuron_steps", 12500)),
    )
    sae.load_state_dict(
        torch.load(OUT_DIR / "sae_model_ultra.pth", map_location="cpu")
    )
    sae.to(device)
    sae.eval()

    dec_w = sae.decoder.weight
    model_dim = ultra.output_layer.weight.shape[1]
    n_feats = (
        int(dec_w.shape[1])
        if int(dec_w.shape[0]) == int(model_dim)
        else int(dec_w.shape[0])
    )
    print(
        f"SAE: input_dim={sae_cfg.get('input_dim')} | "
        f"expansion={sae_cfg.get('expansion_factor')} | "
        f"feats≈{n_feats}"
    )

    # Probe mode: capture SAE activations without changing model outputs.
    hook, handle = register_hooks(
        ultra,
        sae_model=sae,
        bypass=False,
        no_change=True,
    )

    try:
        vocab_size = len(vocab)
        inv_vocab_list = [inv_vocab[i] for i in range(vocab_size)]
        skip_ids = {vocab.get(PAD), vocab.get(NULL)}

        def _compress_ws(text: str) -> str:
            return " ".join((text or "").strip().split())

        def _wrap_notes(text: str, *, width: int = 80, indent: int = 6) -> str:
            compressed = _compress_ws(text)
            if not compressed:
                return ""
            wrapped = textwrap.fill(compressed, width=width)
            return textwrap.indent(wrapped, " " * indent)

        def load_ultra_feature_labels() -> dict[int, dict[str, str]]:
            candidates = [
                OUT_DIR / "consolidated_ultra.json",
                OUT_DIR / "feature_labels_ultra.json",
            ]
            for path in candidates:
                if not path.exists():
                    continue
                raw = json.loads(path.read_text())
                if not isinstance(raw, dict):
                    continue

                consolidated = any(
                    isinstance(v, dict)
                    and (
                        "feature_id" in v
                        or "display_name" in v
                        or "dashboard_name" in v
                        or "dashboard_notes" in v
                    )
                    for v in raw.values()
                )

                labels: dict[int, dict[str, str]] = {}
                if consolidated:
                    for k, v in raw.items():
                        if not isinstance(v, dict):
                            continue
                        fid = v.get("feature_id")
                        if fid is None:
                            try:
                                fid = int(k)
                            except (TypeError, ValueError):
                                continue
                        labels[int(fid)] = {
                            "name": v.get("display_name")
                            or v.get("dashboard_name")
                            or "",
                            "category": v.get("dashboard_category") or "none",
                            "notes": v.get("dashboard_notes") or "",
                        }
                    return labels

                for k, v in raw.items():
                    if not isinstance(v, dict):
                        continue
                    try:
                        fid = int(k)
                    except ValueError:
                        continue
                    labels[fid] = {
                        "name": v.get("name") or "",
                        "category": v.get("category") or "none",
                        "notes": v.get("notes") or "",
                    }
                return labels

            return {}

        feature_labels = load_ultra_feature_labels()
        if feature_labels:
            print("Loaded feature labels:", len(feature_labels))

        def forward_probs(tokens: list[str], weapon_id: str) -> torch.Tensor:
            ids = [vocab[t] for t in tokens]
            x = torch.tensor([ids], device=device)
            w = torch.tensor([[weapon_vocab[weapon_id]]], device=device)
            mask = x == pad_token_id
            with torch.inference_mode():
                probs = torch.sigmoid(
                    ultra(x, w, key_padding_mask=mask)
                ).squeeze(0)
            return probs.detach().cpu()

        def predict_probs_dict(current_tokens: list[str], weapon_id: str):
            probs = forward_probs(current_tokens, weapon_id).tolist()
            return {inv_vocab_list[i]: float(probs[i]) for i in range(vocab_size)}

        def predict_probs_and_acts(current_tokens: list[str], weapon_id: str):
            probs_dict = predict_probs_dict(current_tokens, weapon_id)
            acts = (
                hook.last_h_post.detach().cpu().flatten()
                if hook.last_h_post is not None
                else None
            )
            return probs_dict, acts

        # Offline reconstruction quality check (does not change model outputs).
        _ = forward_probs([NULL], weapon_id)
        if hook.last_in is not None and hook.last_h_post is not None:
            with torch.inference_mode():
                x_in = hook.last_in.detach()
                h_post = hook.last_h_post.detach()
                x_recon = sae.decode(h_post)
                mse = F.mse_loss(x_recon, x_in).item()
                cos = F.cosine_similarity(
                    x_recon.flatten(),
                    x_in.flatten(),
                    dim=0,
                ).item()
            print("SAE recon (offline) MSE:", float(mse))
            print("SAE recon (offline) cosine:", float(cos))

        # Run a traced beam search in probe mode so we can see activations.
        builds, traces = reconstruct_build(
            predict_fn=predict_probs_and_acts,
            weapon_id=weapon_id,
            initial_context=[NULL],
            allocator=Allocator(),
            beam_size=5,
            max_steps=6,
            top_k=1,
            record_traces=True,
        )

        if not builds or not traces:
            raise RuntimeError("No build/trace produced")

        print("\nUltra build (probe mode):")
        print(builds[0].to_dict())

        acts = traces[0][-1].activations
        if acts is None:
            raise RuntimeError("No SAE activations captured")

        SHOW_LABEL_NOTES = False
        MAX_NOTE_CHARS = 220

        topk = torch.topk(acts, k=10)
        print("\nTop active SAE features (probe mode):")
        for i, v in zip(topk.indices.tolist(), topk.values.tolist()):
            fid = int(i)
            val = float(v)
            meta = feature_labels.get(fid)
            if not meta:
                print(fid, val)
                continue
            cat = (meta.get("category") or "none").strip()
            name = (meta.get("name") or "").strip()
            label = f"[{cat}] {name}".strip() if name else f"[{cat}]"
            print(f"{fid:5d} {val:.4f} {label}")
            if SHOW_LABEL_NOTES:
                notes = _wrap_notes(meta.get("notes", ""))
                if notes:
                    if len(notes) > MAX_NOTE_CHARS:
                        notes = notes[: MAX_NOTE_CHARS - 1] + "…"
                    print(notes)

        def pick_feature(feature_ids: list[int]) -> int:
            preferred: list[int] = []
            fallback: list[int] = []
            for fid in feature_ids:
                meta = feature_labels.get(int(fid))
                if not meta:
                    continue
                name = (meta.get("name") or "").strip()
                cat = (meta.get("category") or "").strip().lower()
                if not name:
                    continue
                if cat in {"strategic", "tactical"}:
                    preferred.append(int(fid))
                else:
                    fallback.append(int(fid))
            if preferred:
                return preferred[0]
            if fallback:
                return fallback[0]
            return int(feature_ids[0])

        feature_id = pick_feature([int(i) for i in topk.indices.tolist()])
        feature_value = float(acts[feature_id])
        print(f"\nExample feature: {feature_id} (value={feature_value:.4f})")

        dec_w = sae.decoder.weight
        model_dim = ultra.output_layer.weight.shape[1]
        if int(dec_w.shape[0]) != int(model_dim):
            dec_w = dec_w.T

        decoder_norm = F.normalize(dec_w, dim=0)
        influence = ultra.output_layer.weight @ decoder_norm[:, feature_id]
        influence = influence.detach().cpu().tolist()

        infl_pairs = []
        for i, val in enumerate(influence):
            tok = inv_vocab_list[i]
            if tok.startswith("<") or i in skip_ids:
                continue
            infl_pairs.append((tok, float(val)))
        infl_pairs.sort(key=lambda kv: kv[1], reverse=True)

        print("Top + tokens by logit influence:")
        for tok, val in infl_pairs[:10]:
            print(f"{tok:<32} {val:+.4f}")

        print("\nTop - tokens by logit influence:")
        for tok, val in sorted(infl_pairs, key=lambda kv: kv[1])[:10]:
            print(f"{tok:<32} {val:+.4f}")

    finally:
        handle.remove()
